In [1]:
import copernicusmarine
import xarray
import pandas as pd
import pickle
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
DS = copernicusmarine.open_dataset(dataset_id="C3S-GLO-SST-L4-REP-OBS-SST", username="sjohnson3", password="Soccerisawsome88**")
print(DS.info)


INFO - 2024-12-09T09:19:04Z - Dataset version was not specified, the latest one was selected: "202211"
INFO - 2024-12-09T09:19:04Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-12-09T09:19:05Z - Service was not specified, the default one was selected: "arco-geo-series"


<bound method Dataset.info of <xarray.Dataset> Size: 2TB
Dimensions:                   (time: 2130, latitude: 3600, longitude: 7200,
                               bnds: 2)
Coordinates:
    lat_bnds                  (latitude, bnds) float32 29kB ...
  * latitude                  (latitude) float32 14kB -89.97 -89.93 ... 89.97
    lon_bnds                  (longitude, bnds) float32 58kB ...
  * longitude                 (longitude) float32 29kB -180.0 -179.9 ... 180.0
  * time                      (time) datetime64[ns] 17kB 2017-01-01 ... 2022-...
    time_bnds                 (bnds) datetime64[ns] 16B ...
Dimensions without coordinates: bnds
Data variables:
    analysed_sst              (time, latitude, longitude) float64 442GB ...
    analysed_sst_uncertainty  (time, latitude, longitude) float64 442GB ...
    mask                      (time, latitude, longitude) float32 221GB ...
    sea_ice_fraction          (time, latitude, longitude) float64 442GB ...
Attributes: (12/60)
    Conven

In [43]:
subset = DS[["analysed_sst"]].sel(latitude=slice(5,35), longitude=slice(-180,-120), time=slice("2021-01-01", "2021-12-31"))
df = subset["analysed_sst"].to_dataframe().dropna()

In [45]:
averaged_sst = df["analysed_sst"].groupby(level=["latitude", "longitude"]).mean()
print(averaged_sst)

latitude   longitude  
5.025000   -179.975006    301.667144
           -179.925003    301.666596
           -179.875000    301.661254
           -179.824997    301.654843
           -179.774994    301.648021
                             ...    
34.974998  -120.875000    286.063994
           -120.824997    285.916624
           -120.775002    285.846706
           -120.724998    285.816816
           -120.675003    285.879774
Name: analysed_sst, Length: 719420, dtype: float64


In [47]:
print(averaged_sst.mean())
print(averaged_sst.var())


297.54487780699736
8.917391980058117


In [48]:
subset2 = DS[["analysed_sst"]].sel(latitude=slice(5,10), longitude=slice(-125,-120), time=slice("2021-01-01", "2021-12-31"))
df2 = subset["analysed_sst"].to_dataframe().dropna()

In [50]:
pd.concat([df, df2]).mean()

analysed_sst    297.544878
dtype: float64

In [69]:
dataset = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_phy-all_my_0.25deg_P1M-m", username="sjohnson3", password="Soccerisawsome88**")

INFO - 2024-12-09T10:33:49Z - Dataset version was not specified, the latest one was selected: "202311"
INFO - 2024-12-09T10:33:49Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-12-09T10:33:50Z - Service was not specified, the default one was selected: "arco-geo-series"


In [74]:
dataset.info
dataset.data_vars

Data variables:
    mlotst_cglo   (time, latitude, longitude) float32 1GB ...
    mlotst_glor   (time, latitude, longitude) float32 1GB ...
    mlotst_oras   (time, latitude, longitude) float32 1GB ...
    siconc_cglo   (time, latitude, longitude) float32 1GB ...
    siconc_glor   (time, latitude, longitude) float32 1GB ...
    siconc_oras   (time, latitude, longitude) float32 1GB ...
    sithick_cglo  (time, latitude, longitude) float32 1GB ...
    sithick_glor  (time, latitude, longitude) float32 1GB ...
    sithick_oras  (time, latitude, longitude) float32 1GB ...
    so_cglo       (time, depth, latitude, longitude) float32 109GB ...
    so_glor       (time, depth, latitude, longitude) float32 109GB ...
    so_oras       (time, depth, latitude, longitude) float32 109GB ...
    thetao_cglo   (time, depth, latitude, longitude) float32 109GB ...
    thetao_glor   (time, depth, latitude, longitude) float32 109GB ...
    thetao_oras   (time, depth, latitude, longitude) float32 109GB ...


In [75]:
subset3 = dataset[["thetao_cglo"]].isel(depth=0).sel(latitude=slice(5,10), longitude=slice(-125,-120), time=slice("2004-01-01", "2005-01-01"))

In [76]:
df3 = subset3["thetao_cglo"].to_dataframe().dropna()
print(df3)

                                 depth  thetao_cglo
time       latitude longitude                      
2004-01-01 5.0      -125.00    0.50576    27.127182
                    -124.75    0.50576    27.082748
                    -124.50    0.50576    27.093428
                    -124.25    0.50576    27.137844
                    -124.00    0.50576    27.169683
...                                ...          ...
2005-01-01 10.0     -121.00    0.50576    27.275768
                    -120.75    0.50576    27.266563
                    -120.50    0.50576    27.258965
                    -120.25    0.50576    27.262264
                    -120.00    0.50576    27.269365

[5733 rows x 2 columns]


In [79]:
subset4 = dataset[["thetao_cglo"]].isel(depth=0).sel(latitude=slice(10,15), longitude=slice(-125,-120), time=slice("2004-01-01", "2005-01-01"))

In [80]:
df4 = subset4["thetao_cglo"].to_dataframe().dropna()
pd.concat([df3, df4])

depth  thetao_cglo
time       latitude longitude                      
2004-01-01 5.0      -125.00    0.50576    27.127182
                    -124.75    0.50576    27.082748
                    -124.50    0.50576    27.093428
                    -124.25    0.50576    27.137844
                    -124.00    0.50576    27.169683
...                                ...          ...
2005-01-01 15.0     -121.00    0.50576    25.232134
                    -120.75    0.50576    25.391127
                    -120.50    0.50576    25.540783
                    -120.25    0.50576    25.673635
                    -120.00    0.50576    25.801678

[11466 rows x 2 columns]

In [122]:
area_coords = {
    21: {"lat": [slice(35, 78)], "long": [slice(-80, -42)]},
    27: {
        "lat": [slice(36, 90), slice(43, 90), slice(49, 90)],
        "long": [slice(-42, -6), slice(-6, 0), slice(0, 68)],
    },
    31: {
        "lat": [slice(17, 35), slice(15, 17), slice(8.4, 15), slice(5, 8.4)],
        "long": [slice(-98, -40), slice(-90, -40), slice(-84, -40), slice(-63, -40)],
    },
    34: {"lat": [slice(-6, 36), slice(-6, 8)], "long": [slice(-20, -5), slice(-5, 14)]},
    37: {"lat": [slice(34, 42), slice(30, 47)], "long": [slice(-5, 3), slice(3, 42)]},
    41: {
        "lat": [slice(-60, -34), slice(-50, 5)],
        "long": [slice(-67, -54), slice(-54, -20)],
    },
    47: {"lat": [slice(-40, -6)], "long": [slice(5, 30)]},
    51: {"lat": [slice(-45, 30)], "long": [slice(30, 77)]},
    57: {
        "lat": [slice(-55, 24), slice(-55, -1), slice(-55, -8), slice(-55, -30)],
        "long": [slice(77, 100), slice(100, 105), slice(105, 129), slice(129, 150)],
    },
    61: {
        "lat": [slice(15, 23), slice(20, 65)],
        "long": [slice(105, 115), slice(115, 180)],
    },
    67: {"lat": [slice(40, 66)], "long": [slice(-175, -123)]},
    71: {
        "lat": [slice(-8, 15), slice(15, 20), slice(-25, -8)],
        "long": [slice(99, 129), slice(115, 175), slice(129, 175)],
    },
    77: {
        "lat": [slice(-25, 40), slice(5, 40), slice(5, 15), slice(5, 8)],
        "long": [slice(-175, -120), slice(-120, -98), slice(-98, -84), slice(-84, -78)],
    },
    81: {
        "lat": [slice(-60, -25), slice(-60, -25)],
        "long": [slice(150, 180), slice(-180, -120)],
    },
    87: {
        "lat": [slice(-60, -55), slice(-55, 5)],
        "long": [slice(-120, -67), slice(-120, -69)],
    },
}

years = {
    2004: slice("2004-01-01", "2004-12-31"),
    # 2006: slice("2006-01-01", "2006-12-31"),
    # 2008: slice("2008-01-01", "2008-12-31"),
    # 2009: slice("2009-01-01", "2009-12-31"),
    # 2011: slice("2011-01-01", "2011-12-31"),
    # 2013: slice("2013-01-01", "2013-12-31"),
    # 2015: slice("2015-01-01", "2015-12-31"),
    # 2017: slice("2017-01-01", "2017-12-31"),
    # 2019: slice("2019-01-01", "2019-12-31"),
    # 2021: slice("2021-01-01", "2021-12-31"),
}

data_template = {year: pd.DataFrame() for year in years.keys()}

In [96]:
def build_data(dataset, pickle_file, variable, depth=0):
    print(f"Loading data for {variable}")

    with open(pickle_file, "rb") as file:
        data = pickle.load(file)

    for key in area_coords.keys():
        data[key][variable] = data_template

    for area, coords in area_coords.items():
        for year, year_slice in years.items():
            for lat, long in zip(coords["lat"], coords["long"]):
                if depth is not None:
                    subset = (
                        dataset[[variable]]
                        .isel(depth=depth)
                        .sel(latitude=lat, longitude=long, time=year_slice)
                    )
                else:
                    subset = dataset[[variable]].sel(
                        latitude=lat, longitude=long, time=year_slice
                    )
                data[area][variable][year] = pd.concat(
                    [
                        data[area][variable][year],
                        subset[variable].to_dataframe().dropna(),
                    ]
                )
            print(f"Completed year {year} for area {area}")

    with open(pickle_file, "wb") as file:
        pickle.dump(data, file)

    print(f"{variable} data loaded onto {pickle_file}")

In [97]:
dataset1 = copernicusmarine.open_dataset(
    dataset_id="cmems_mod_glo_phy-all_my_0.25deg_P1M-m",
    username="sjohnson3",
    password="Soccerisawsome88**",
)

INFO - 2024-12-10T08:22:42Z - Dataset version was not specified, the latest one was selected: "202311"
INFO - 2024-12-10T08:22:42Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-12-10T08:22:43Z - Service was not specified, the default one was selected: "arco-geo-series"


In [120]:
oceanography_data = {area: {} for area in area_coords.keys()}

with open("oceanography_data.pkl", "wb") as file:
    pickle.dump(oceanography_data, file)

In [99]:
build_data(dataset1, "oceanography_data.pkl", "thetao_cglo")

Loading data for thetao_cglo
Completed year 2004 for area 21
Completed year 2004 for area 27
Completed year 2004 for area 31
Completed year 2004 for area 34
Completed year 2004 for area 37
Completed year 2004 for area 41
Completed year 2004 for area 47
Completed year 2004 for area 51
Completed year 2004 for area 57
Completed year 2004 for area 61
Completed year 2004 for area 67
Completed year 2004 for area 71
Completed year 2004 for area 77
Completed year 2004 for area 81
Completed year 2004 for area 87
thetao_cglo data loaded onto oceanography_data.pkl


In [100]:
with open("oceanography_data.pkl", "rb") as file:
    data = pickle.load(file)

In [118]:
ex = data[21]["thetao_cglo"][2004]

ave = ex.groupby("time")["thetao_cglo"].mean().reset_index()

print(ave)

         time  thetao_cglo
0  2004-01-01    17.696514
1  2004-02-01    17.808388
2  2004-03-01    17.788788
3  2004-04-01    17.766041
4  2004-05-01    17.678808
5  2004-06-01    17.754772
6  2004-07-01    18.022060
7  2004-08-01    18.230362
8  2004-09-01    18.082628
9  2004-10-01    17.779190
10 2004-11-01    17.594946
11 2004-12-01    17.601620


In [123]:
def build_data2(dataset, pickle_file, variable, depth=0):
    print(f"Loading data for {variable}")

    with open(pickle_file, "rb") as file:
        data = pickle.load(file)

    for key in area_coords.keys():
        data[key][variable] = data_template

    for area, coords in area_coords.items():
        for year, year_slice in years.items():
            for lat, long in zip(coords["lat"], coords["long"]):
                if depth is not None:
                    subset = (
                        dataset[[variable]]
                        .isel(depth=depth)
                        .sel(latitude=lat, longitude=long, time=year_slice)
                    )
                else:
                    subset = dataset[[variable]].sel(
                        latitude=lat, longitude=long, time=year_slice
                    )
                data[area][variable][year] = pd.concat(
                    [
                        data[area][variable][year],
                        subset[variable].to_dataframe().dropna(),
                    ]
                )

            data[area][variable][year] = data[area][variable][year].groupby("time")[variable].mean().reset_index()
            print(f"Completed year {year} for area {area}")

    with open(pickle_file, "wb") as file:
        pickle.dump(data, file)

    print(f"{variable} data loaded onto {pickle_file}")

In [124]:
build_data2(dataset1, "oceanography_data.pkl", "thetao_cglo")

Loading data for thetao_cglo
Completed year 2004 for area 21
Completed year 2004 for area 27
Completed year 2004 for area 31
Completed year 2004 for area 34
Completed year 2004 for area 37
Completed year 2004 for area 41
Completed year 2004 for area 47
Completed year 2004 for area 51
Completed year 2004 for area 57
Completed year 2004 for area 61
Completed year 2004 for area 67
Completed year 2004 for area 71
Completed year 2004 for area 77
Completed year 2004 for area 81
Completed year 2004 for area 87
thetao_cglo data loaded onto oceanography_data.pkl


In [140]:
with open("oceanography_data.pkl", "rb") as file:
    data = pickle.load(file)

In [141]:
print(data)

{21: {'thetao_mean': {2004:          time  thetao_mean
0  2004-01-01     3.592387
1  2004-02-01     3.370120
2  2004-03-01     3.312194
3  2004-04-01     3.551373
4  2004-05-01     4.169974
5  2004-06-01     5.360918
6  2004-07-01     6.843750
7  2004-08-01     7.654229
8  2004-09-01     7.060114
9  2004-10-01     5.809432
10 2004-11-01     4.745907
11 2004-12-01     4.114413}, 'so_mean': {2004:          time    so_mean
0  2004-01-01  33.743935
1  2004-02-01  33.834263
2  2004-03-01  33.880486
3  2004-04-01  33.901989
4  2004-05-01  33.806297
5  2004-06-01  33.345737
6  2004-07-01  32.673450
7  2004-08-01  32.734409
8  2004-09-01  33.052258
9  2004-10-01  33.281876
10 2004-11-01  33.518410
11 2004-12-01  33.703823}}, 27: {'thetao_mean': {2004:          time  thetao_mean
0  2004-01-01     3.592387
1  2004-02-01     3.370120
2  2004-03-01     3.312194
3  2004-04-01     3.551373
4  2004-05-01     4.169974
5  2004-06-01     5.360918
6  2004-07-01     6.843750
7  2004-08-01     7.654229
8  

In [3]:
with open("oceanography_data.pkl", "rb") as file:
    oceanography_data = pickle.load(file)

dict_keys([21, 27, 31, 34, 37, 41, 47, 51, 57, 61, 67, 71, 77, 81, 87])
